### This is a test project to predict daily SP500 using AlphaVantage (AV) API for datapull

In [55]:
import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests, optuna
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft


from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

time0 = time.time()

os.chdir('/home/jupyter/projects_gcp_cpu/spx/src')
os.getcwd()

'/home/jupyter/projects_gcp_cpu/spx/src'

Datapull using AV APIs:

In [2]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=SPY&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
# r = requests.get(url)
# data = r.json()
spy = pd.read_csv(url)
display(spy.shape, spy.head())
spy = spy[['timestamp', 'adjusted_close']]
display(spy.shape, spy.head())


(5804, 9)

,timestamp,open,high,low,close,adjusted_close,volume,dividend_amount,split_coefficient
0,2022-11-22,396.63,400.07,395.15,399.90,399.90,60062763,0.0,1.0
1,2022-11-21,394.64,395.82,392.66,394.59,394.59,51243189,0.0,1.0
2,2022-11-18,397.74,397.81,393.04,396.03,396.03,92922451,0.0,1.0
3,2022-11-17,390.46,394.95,390.14,394.24,394.24,74496255,0.0,1.0
4,2022-11-16,396.78,397.78,394.79,395.45,395.45,68508454,0.0,1.0


(5804, 2)

,timestamp,adjusted_close
0,2022-11-22,399.90
1,2022-11-21,394.59
2,2022-11-18,396.03
3,2022-11-17,394.24
4,2022-11-16,395.45


In [3]:
url_1yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=1year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr1 = pd.read_csv(url_1yr)
tr1.rename(columns={'value':'yield_1y'}, inplace=True)

url_10yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=10year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr10 = pd.read_csv(url_10yr)
tr10.rename(columns={'value':'yield_10y'}, inplace=True)

url_30yr = 'https://www.alphavantage.co/query?function=TREASURY_YIELD&interval=daily&maturity=30year&apikey=KBYUCPQEHAG67WNC&datatype=csv&outputsize=full'
tr30 = pd.read_csv(url_30yr)
tr30.rename(columns={'value':'yield_30y'}, inplace=True)

display(tr30)

rates = ft.reduce(lambda left, right: pd.merge(left, right, on='timestamp'), [tr1, tr10, tr30])
display(rates)

,timestamp,yield_30y
0,2022-11-18,3.92
1,2022-11-17,3.89
2,2022-11-16,3.85
3,2022-11-15,3.98
4,2022-11-14,4.07
...,...,...
11934,1977-02-21,.
11935,1977-02-18,7.76
11936,1977-02-17,7.67
11937,1977-02-16,7.67


,timestamp,yield_1y,yield_10y,yield_30y
0,2022-11-18,3.82,3.82,3.92
1,2022-11-17,3.77,3.77,3.89
2,2022-11-16,3.67,3.67,3.85
3,2022-11-15,3.80,3.80,3.98
4,2022-11-14,3.88,3.88,4.07
...,...,...,...,...
11934,1977-02-21,.,.,.
11935,1977-02-18,7.41,7.41,7.76
11936,1977-02-17,7.26,7.26,7.67
11937,1977-02-16,7.34,7.34,7.67


In [4]:
df = pd.merge(spy, rates, on='timestamp')
df.rename(columns = {'adjusted_close':'spx'}, inplace=True)
df.spx = df.spx*10
display(df.info(), df.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5802 entries, 0 to 5801
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  5802 non-null   object 
 1   spx        5802 non-null   float64
 2   yield_1y   5802 non-null   object 
 3   yield_10y  5802 non-null   object 
 4   yield_30y  5802 non-null   object 
dtypes: float64(1), object(4)
memory usage: 272.0+ KB


None

,timestamp,spx,yield_1y,yield_10y,yield_30y
0,2022-11-18,3960.3,3.82,3.82,3.92
1,2022-11-17,3942.4,3.77,3.77,3.89
2,2022-11-16,3954.5,3.67,3.67,3.85
3,2022-11-15,3984.9,3.80,3.80,3.98
4,2022-11-14,3951.2,3.88,3.88,4.07


In [5]:
rate_cols = ['yield_1y', 'yield_10y', 'yield_30y']
df[rate_cols] = df[rate_cols].apply(pd.to_numeric, errors='coerce')
df.fillna(method='ffill', inplace=True)
display(df.info(), df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5802 entries, 0 to 5801
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   timestamp  5802 non-null   object 
 1   spx        5802 non-null   float64
 2   yield_1y   5802 non-null   float64
 3   yield_10y  5802 non-null   float64
 4   yield_30y  5802 non-null   float64
dtypes: float64(4), object(1)
memory usage: 272.0+ KB


None

,timestamp,spx,yield_1y,yield_10y,yield_30y
0,2022-11-18,3960.300000,3.82,3.82,3.92
1,2022-11-17,3942.400000,3.77,3.77,3.89
2,2022-11-16,3954.500000,3.67,3.67,3.85
3,2022-11-15,3984.900000,3.80,3.80,3.98
4,2022-11-14,3951.200000,3.88,3.88,4.07
...,...,...,...,...,...
5797,1999-11-05,904.871930,5.92,5.92,6.05
5798,1999-11-04,896.052613,5.95,5.95,6.09
5799,1999-11-03,889.284834,6.01,6.01,6.14
5800,1999-11-02,883.336825,6.04,6.04,6.15


In [6]:
df['date'] = pd.to_datetime(df.timestamp)
df.sort_values(by='date', inplace=True)
df.drop(columns='timestamp', inplace=True)
df

,spx,yield_1y,yield_10y,yield_30y,date
5801,889.695021,6.06,6.06,6.19,1999-11-01
5800,883.336825,6.04,6.04,6.15,1999-11-02
5799,889.284834,6.01,6.01,6.14,1999-11-03
5798,896.052613,5.95,5.95,6.09,1999-11-04
5797,904.871930,5.92,5.92,6.05,1999-11-05
...,...,...,...,...,...
4,3951.200000,3.88,3.88,4.07,2022-11-14
3,3984.900000,3.80,3.80,3.98,2022-11-15
2,3954.500000,3.67,3.67,3.85,2022-11-16
1,3942.400000,3.77,3.77,3.89,2022-11-17


In [7]:
laggs = [1, 5 , 21, 63, 126, 252, 252*2, 252*3]

for lg in laggs:
    df['spx_l' + str(lg)] = df.spx.shift(lg)
    df['ty1y_l' + str(lg)] = df.yield_1y.shift(lg)
    df['ty10y_l' + str(lg)] = df.yield_10y.shift(lg)
    df['ty30y_l' + str(lg)] = df.yield_30y.shift(lg)
    
    df['spx_ret_' + str(lg)] = (df.spx/df['spx_l' + str(lg)])-1
    df['ty1y_diff_' + str(lg)] = (df.yield_1y-df['ty1y_l' + str(lg)])
    df['ty10y_diff_' + str(lg)] = (df.yield_10y-df['ty10y_l' + str(lg)])
    df['ty30y_diff_' + str(lg)] = (df.yield_30y-df['ty30y_l' + str(lg)])

colstokeep = ['date', 'spx', 'yield_1y', 'yield_10y', 'yield_30y'] + \
            ['spx_ret_' + str(x) for x in laggs] + \
            ['ty1y_diff_' + str(x) for x in laggs] + \
            ['ty10y_diff_' + str(x) for x in laggs] + \
            ['ty30y_diff_' + str(x) for x in laggs] 

df = df[colstokeep]

df.head(8)

,date,spx,yield_1y,yield_10y,yield_30y,spx_ret_1,spx_ret_5,spx_ret_21,spx_ret_63,spx_ret_126,spx_ret_252,spx_ret_504,spx_ret_756,ty1y_diff_1,ty1y_diff_5,ty1y_diff_21,ty1y_diff_63,ty1y_diff_126,ty1y_diff_252,ty1y_diff_504,ty1y_diff_756,ty10y_diff_1,ty10y_diff_5,ty10y_diff_21,ty10y_diff_63,ty10y_diff_126,ty10y_diff_252,ty10y_diff_504,ty10y_diff_756,ty30y_diff_1,ty30y_diff_5,ty30y_diff_21,ty30y_diff_63,ty30y_diff_126,ty30y_diff_252,ty30y_diff_504,ty30y_diff_756
5801,1999-11-01,889.695021,6.06,6.06,6.19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5800,1999-11-02,883.336825,6.04,6.04,6.15,-0.007146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5799,1999-11-03,889.284834,6.01,6.01,6.14,0.006734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5798,1999-11-04,896.052613,5.95,5.95,6.09,0.007610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5797,1999-11-05,904.871930,5.92,5.92,6.05,0.009842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5796,1999-11-08,905.692304,5.95,5.95,6.06,0.000907,0.017981,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,0.01,-0.13,NaN,NaN,NaN,NaN,NaN,NaN
5795,1999-11-09,897.180725,5.97,5.97,6.07,-0.009398,0.015672,NaN,NaN,NaN,NaN,NaN,NaN,0.02,-0.07,NaN,NaN,NaN,NaN,NaN,NaN,0.02,-0.07,NaN,NaN,NaN,NaN,NaN,NaN,0.01,-0.08,NaN,NaN,NaN,NaN,NaN,NaN
5794,1999-11-10,903.846161,6.00,6.00,6.09,0.007429,0.016374,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,0.02,-0.05,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df0 = df.copy()
df['target'] = df.ty10y_diff_1
df = df[df['target'].notna()]
df.drop(columns = ['spx', 'yield_1y', 'yield_10y', 'yield_30y'], inplace=True)
df

,date,spx_ret_1,spx_ret_5,spx_ret_21,spx_ret_63,spx_ret_126,spx_ret_252,spx_ret_504,spx_ret_756,ty1y_diff_1,ty1y_diff_5,ty1y_diff_21,ty1y_diff_63,ty1y_diff_126,ty1y_diff_252,ty1y_diff_504,ty1y_diff_756,ty10y_diff_1,ty10y_diff_5,ty10y_diff_21,ty10y_diff_63,ty10y_diff_126,ty10y_diff_252,ty10y_diff_504,ty10y_diff_756,ty30y_diff_1,ty30y_diff_5,ty30y_diff_21,ty30y_diff_63,ty30y_diff_126,ty30y_diff_252,ty30y_diff_504,ty30y_diff_756,target
5800,1999-11-02,-0.007146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02
5799,1999-11-03,0.006734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
5798,1999-11-04,0.007610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06
5797,1999-11-05,0.009842,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
5796,1999-11-08,0.000907,0.017981,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,0.03,-0.11,NaN,NaN,NaN,NaN,NaN,NaN,0.01,-0.13,NaN,NaN,NaN,NaN,NaN,NaN,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-11-14,-0.008507,0.039926,0.104829,-0.076667,-0.004059,-0.141588,0.151724,0.341951,0.00,-0.34,-0.12,1.06,1.00,2.30,3.00,2.00,0.00,-0.34,-0.12,1.06,1.00,2.30,3.00,2.00,0.00,-0.27,0.08,0.96,0.98,2.12,2.43,1.71,0.00
3,2022-11-15,0.008529,0.043168,0.086337,-0.062135,-0.015810,-0.134563,0.145686,0.351429,-0.08,-0.34,-0.22,0.91,0.82,2.17,2.91,1.98,-0.08,-0.34,-0.22,0.91,0.82,2.17,2.91,1.98,-0.09,-0.30,-0.06,0.83,0.81,1.97,2.33,1.67,-0.08
2,2022-11-16,-0.007629,0.056986,0.065530,-0.071987,0.017707,-0.144551,0.122929,0.331484,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.46,-0.19,0.71,0.78,1.83,2.19,1.54,-0.13
1,2022-11-17,-0.003060,-0.001140,0.069851,-0.062225,0.020846,-0.145091,0.125546,0.326432,0.10,-0.05,-0.37,0.79,0.93,2.17,2.90,1.96,0.10,-0.05,-0.37,0.79,0.93,2.17,2.90,1.96,0.04,-0.14,-0.26,0.67,0.84,1.89,2.27,1.59,0.10


In [9]:
cols_tolag = [col for col in df.columns if col not in ['date', 'target']]
cols_tolag

for col in cols_tolag:
    df[col] = df[col].shift(1)
    
display(df.head(4), df.tail(2))

,date,spx_ret_1,spx_ret_5,spx_ret_21,spx_ret_63,spx_ret_126,spx_ret_252,spx_ret_504,spx_ret_756,ty1y_diff_1,ty1y_diff_5,ty1y_diff_21,ty1y_diff_63,ty1y_diff_126,ty1y_diff_252,ty1y_diff_504,ty1y_diff_756,ty10y_diff_1,ty10y_diff_5,ty10y_diff_21,ty10y_diff_63,ty10y_diff_126,ty10y_diff_252,ty10y_diff_504,ty10y_diff_756,ty30y_diff_1,ty30y_diff_5,ty30y_diff_21,ty30y_diff_63,ty30y_diff_126,ty30y_diff_252,ty30y_diff_504,ty30y_diff_756,target
5800,1999-11-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02
5799,1999-11-03,-0.007146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03
5798,1999-11-04,0.006734,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06
5797,1999-11-05,0.007610,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.03


,date,spx_ret_1,spx_ret_5,spx_ret_21,spx_ret_63,spx_ret_126,spx_ret_252,spx_ret_504,spx_ret_756,ty1y_diff_1,ty1y_diff_5,ty1y_diff_21,ty1y_diff_63,ty1y_diff_126,ty1y_diff_252,ty1y_diff_504,ty1y_diff_756,ty10y_diff_1,ty10y_diff_5,ty10y_diff_21,ty10y_diff_63,ty10y_diff_126,ty10y_diff_252,ty10y_diff_504,ty10y_diff_756,ty30y_diff_1,ty30y_diff_5,ty30y_diff_21,ty30y_diff_63,ty30y_diff_126,ty30y_diff_252,ty30y_diff_504,ty30y_diff_756,target
1,2022-11-17,-0.007629,0.056986,0.065530,-0.071987,0.017707,-0.144551,0.122929,0.331484,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.46,-0.19,0.71,0.78,1.83,2.19,1.54,0.10
0,2022-11-18,-0.003060,-0.001140,0.069851,-0.062225,0.020846,-0.145091,0.125546,0.326432,0.10,-0.05,-0.37,0.79,0.93,2.17,2.90,1.96,0.10,-0.05,-0.37,0.79,0.93,2.17,2.90,1.96,0.04,-0.14,-0.26,0.67,0.84,1.89,2.27,1.59,0.05


In [10]:
display(df.info())
df.fillna(value=0.0, inplace=True)
display(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5801 entries, 5800 to 0
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            5801 non-null   datetime64[ns]
 1   spx_ret_1       5800 non-null   float64       
 2   spx_ret_5       5796 non-null   float64       
 3   spx_ret_21      5780 non-null   float64       
 4   spx_ret_63      5738 non-null   float64       
 5   spx_ret_126     5675 non-null   float64       
 6   spx_ret_252     5549 non-null   float64       
 7   spx_ret_504     5297 non-null   float64       
 8   spx_ret_756     5045 non-null   float64       
 9   ty1y_diff_1     5800 non-null   float64       
 10  ty1y_diff_5     5796 non-null   float64       
 11  ty1y_diff_21    5780 non-null   float64       
 12  ty1y_diff_63    5738 non-null   float64       
 13  ty1y_diff_126   5675 non-null   float64       
 14  ty1y_diff_252   5549 non-null   float64       
 15  ty1y

None

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5801 entries, 5800 to 0
Data columns (total 34 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   date            5801 non-null   datetime64[ns]
 1   spx_ret_1       5801 non-null   float64       
 2   spx_ret_5       5801 non-null   float64       
 3   spx_ret_21      5801 non-null   float64       
 4   spx_ret_63      5801 non-null   float64       
 5   spx_ret_126     5801 non-null   float64       
 6   spx_ret_252     5801 non-null   float64       
 7   spx_ret_504     5801 non-null   float64       
 8   spx_ret_756     5801 non-null   float64       
 9   ty1y_diff_1     5801 non-null   float64       
 10  ty1y_diff_5     5801 non-null   float64       
 11  ty1y_diff_21    5801 non-null   float64       
 12  ty1y_diff_63    5801 non-null   float64       
 13  ty1y_diff_126   5801 non-null   float64       
 14  ty1y_diff_252   5801 non-null   float64       
 15  ty1y

None

In [11]:
df

,date,spx_ret_1,spx_ret_5,spx_ret_21,spx_ret_63,spx_ret_126,spx_ret_252,spx_ret_504,spx_ret_756,ty1y_diff_1,ty1y_diff_5,ty1y_diff_21,ty1y_diff_63,ty1y_diff_126,ty1y_diff_252,ty1y_diff_504,ty1y_diff_756,ty10y_diff_1,ty10y_diff_5,ty10y_diff_21,ty10y_diff_63,ty10y_diff_126,ty10y_diff_252,ty10y_diff_504,ty10y_diff_756,ty30y_diff_1,ty30y_diff_5,ty30y_diff_21,ty30y_diff_63,ty30y_diff_126,ty30y_diff_252,ty30y_diff_504,ty30y_diff_756,target
5800,1999-11-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.02
5799,1999-11-03,-0.007146,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.02,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.03
5798,1999-11-04,0.006734,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.01,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.06
5797,1999-11-05,0.007610,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.06,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.05,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.03
5796,1999-11-08,0.009842,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.03,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4,2022-11-14,0.009678,0.058881,0.088914,-0.066922,0.000410,-0.127689,0.150337,0.353903,0.06,-0.29,-0.09,1.09,0.95,2.30,3.00,1.96,0.06,-0.29,-0.09,1.09,0.95,2.30,3.00,1.96,0.04,-0.20,0.10,0.97,0.97,2.12,2.43,1.68,0.00
3,2022-11-15,-0.008507,0.039926,0.104829,-0.076667,-0.004059,-0.141588,0.151724,0.341951,0.00,-0.34,-0.12,1.06,1.00,2.30,3.00,2.00,0.00,-0.34,-0.12,1.06,1.00,2.30,3.00,2.00,0.00,-0.27,0.08,0.96,0.98,2.12,2.43,1.71,-0.08
2,2022-11-16,0.008529,0.043168,0.086337,-0.062135,-0.015810,-0.134563,0.145686,0.351429,-0.08,-0.34,-0.22,0.91,0.82,2.17,2.91,1.98,-0.08,-0.34,-0.22,0.91,0.82,2.17,2.91,1.98,-0.09,-0.30,-0.06,0.83,0.81,1.97,2.33,1.67,-0.13
1,2022-11-17,-0.007629,0.056986,0.065530,-0.071987,0.017707,-0.144551,0.122929,0.331484,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.45,-0.34,0.79,0.78,2.04,2.76,1.83,-0.13,-0.46,-0.19,0.71,0.78,1.83,2.19,1.54,0.10


In [12]:
df.drop(columns = ['date'], inplace=True)
X = df.copy()
y = X.pop('target')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = int(X.shape[0]*0.2))
display(X_train.shape, X_test.shape, y_train.shape)

(4641, 32)

(1160, 32)

(4641,)

In [13]:
y

5800   -0.02
5799   -0.03
5798   -0.06
5797   -0.03
5796    0.03
        ... 
4       0.00
3      -0.08
2      -0.13
1       0.10
0       0.05
Name: target, Length: 5801, dtype: float64

In [14]:
xgbm = XGBRegressor()
xgbm.fit(X_train, y_train)
print(r2_score(y_train, xgbm.predict(X_train)), r2_score(y_test, xgbm.predict(X_test)))

0.8749830373805936 -0.14180419887221185


In [43]:
xgbm = XGBRegressor(n_estimators=200, max_depth=4, eta=0.06, subsample=0.6, colsample_bytree=0.5)
xgbm.fit(X_train, y_train)
print(r2_score(y_train, xgbm.predict(X_train)), r2_score(y_test, xgbm.predict(X_test)))

0.3167750293551599 -0.004575412964100023


In [44]:
adjusted_predictions = y_train.mean() + (xgbm.predict(X_test)-y_train.mean())*0.5
print(r2_score(y_train, xgbm.predict(X_train)), r2_score(y_test, adjusted_predictions))

0.3167750293551599 0.00881135305195846


In [65]:
time1 = time.time()

def objective(trial, cv_runs=1, n_splits=2, n_jobs=-1):

    cv_regularizer=0.001
    # Usually values between 0.1 and 0.2 work fine.

    params = {
        "verbosity": 1,  
        "n_estimators": trial.suggest_int("n_estimators", 100, 400),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
        "learning_rate": trial.suggest_uniform("learning_rate", 0.01, 0.2),
        "colsample_bytree": trial.suggest_uniform("colsample_bytree", 0.1, 0.95),
        "subsample": trial.suggest_uniform("subsample", 0.5, 0.95),
        "alpha": trial.suggest_loguniform("alpha", 0.1, 50.0),
        "lambda": trial.suggest_loguniform("lambda", 0.1, 500.0),
        "gamma": trial.suggest_loguniform("gamma", 1e-10, 50.0),
        "min_child_weight": trial.suggest_loguniform("min_child_weight", 0.1, 10),
        "n_jobs": n_jobs,
    }
    # usually it makes sense to resrtict hyperparameter space from some solutions which Optuna will find
    # e.g., for tmx-joined data only (downsampled tmx), optuna keeps selecting depths of 2 and 3.
    # for my purposes (smooth left side of prc, close to 1), those solutions are no good.

    temp_out = []

    for i in range(cv_runs):

        X = X_train
        y = y_train

        model = XGBRegressor(**params)
        rkf = KFold(n_splits=n_splits, shuffle=True)
        X_values = X.values
        y_values = y.values
        y_pred = np.zeros_like(y_values)
        y_pred_train = np.zeros_like(y_values)
        for train_index, test_index in rkf.split(X_values):
            X_A, X_B = X_values[train_index, :], X_values[test_index, :]
            y_A, y_B = y_values[train_index], y_values[test_index]
            model.fit(X_A, y_A, eval_set=[(X_B, y_B)], verbose = False)
            # y_pred[test_index] = model.predict(X_B).reshape(-1,1)
            y_pred[test_index] = model.predict(X_B)
            y_pred_train[train_index] = model.predict(X_A)
            
        score_train = r2_score(y_train, y_pred_train)
        score_test = r2_score(y_train, y_pred) 
        overfit = score_train-score_test
        temp_out.append(score_test-cv_regularizer*overfit)
        temp_out.append(score_test)

    return (np.mean(temp_out))

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=10)

print('Total time for hypermarameter optimization ', time.time()-time1)
hp = study.best_params
for key, value in hp.items():
    print(f"{key:>20s} : {value}")
print(f"{'best objective value':>20s} : {study.best_value}")

[I 2022-11-22 22:18:38,961] A new study created in memory with name: no-name-2de64521-5cab-4546-8b65-c8cc357c144b
[I 2022-11-22 22:18:43,981] Trial 0 finished with value: -0.008112552332649365 and parameters: {'n_estimators': 383, 'max_depth': 3, 'learning_rate': 0.1994008607288064, 'colsample_bytree': 0.5739018190583927, 'subsample': 0.5754474568292077, 'alpha': 3.707796849741579, 'lambda': 5.687463896331566, 'gamma': 0.00010951051844801794, 'min_child_weight': 0.11786376811453522}. Best is trial 0 with value: -0.008112552332649365.
[I 2022-11-22 22:18:46,864] Trial 1 finished with value: -0.0200329817511598 and parameters: {'n_estimators': 208, 'max_depth': 2, 'learning_rate': 0.05572232987297096, 'colsample_bytree': 0.8251747177174831, 'subsample': 0.682409260243259, 'alpha': 9.86245781185028, 'lambda': 35.46166721943393, 'gamma': 1.5007802799079258e-06, 'min_child_weight': 0.5352642850915612}. Best is trial 0 with value: -0.008112552332649365.
[I 2022-11-22 22:18:51,575] Trial 2 fi

Total time for hypermarameter optimization  40.4836368560791
        n_estimators : 149
           max_depth : 5
       learning_rate : 0.15687268832662787
    colsample_bytree : 0.2799537008084777
           subsample : 0.8249276577106917
               alpha : 1.076586189429261
              lambda : 14.693598483326696
               gamma : 1.3239479034319739
    min_child_weight : 0.6123441776390809
best objective value : -7.91050415806333e-05


In [66]:
optuna_hyperpars = study.best_params
optuna_xgb = XGBRegressor(**optuna_hyperpars, seed=8)
optuna_xgb.fit(X_train, y_train)
print('XGB train', 
      r2_score(y_train, optuna_xgb.predict(X_train)), 
      time.time()-time1)

print('Out of Sample:')
print('XGB ', r2_score(y_test, optuna_xgb.predict(X_test)))
print('Total time ', time.time()-time0)

XGB train -1.6066857846386284e-06 41.202136278152466
Out of Sample:
XGB  -0.0007233217414064352
Total time  339.6874563694


In [ ]:
# can try predicting 10 yield, try smth like xgb 200, 4, 0.04

In [67]:
# it is hard to get any positive results at all using daily freq major assets
# can try to exploit intraday lead-lag effects
# i.e., pick up intraday major assets (3 indices) and try to predict less liquid assets.
# can try btc or etfs of small stocks. e.g., eem or eems etfs. 

In [78]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=SPY&interval=1min&adjusted=true&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
spy = pd.read_csv(url)
spy

,time,open,high,low,close,volume
0,2022-11-21 20:00:00,395.12,395.12,395.10,395.10,852
1,2022-11-21 19:59:00,395.11,395.11,395.11,395.11,1020
2,2022-11-21 19:58:00,395.15,395.19,395.15,395.15,990
3,2022-11-21 19:57:00,395.11,395.20,395.11,395.20,1458
4,2022-11-21 19:56:00,395.10,395.10,395.10,395.10,207
...,...,...,...,...,...,...
18505,2022-10-24 04:05:00,373.28,373.33,373.15,373.15,1986
18506,2022-10-24 04:04:00,372.84,373.11,372.59,373.11,4693
18507,2022-10-24 04:03:00,373.51,373.51,372.96,372.96,4342
18508,2022-10-24 04:02:00,373.51,373.63,373.46,373.63,4383


In [88]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEM&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eem = pd.read_csv(url)
print(eem.shape)

# EEMS is even less liquid, try eema emxc

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEMS&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eems = pd.read_csv(url)
print(eems.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EWX&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
ewx = pd.read_csv(url)
print(ewx.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEMA&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eema = pd.read_csv(url)
print(eema.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EMXC&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
emxc = pd.read_csv(url)
print(emxc.shape)

(9752, 6)
(1210, 6)
(1230, 6)
(1384, 6)
(6033, 6)


In [87]:
display(spy[192:250], eem[:50], emxc[:50])

,time,open,high,low,close,volume
192,2022-11-21 16:01:00,394.5800,394.6900,394.5300,394.6200,195089
193,2022-11-21 16:00:00,394.4900,394.7900,394.4800,394.5800,1210352
194,2022-11-21 15:59:00,394.4250,394.5000,394.3800,394.4900,686851
195,2022-11-21 15:58:00,394.4400,394.5000,394.3900,394.4250,255209
196,2022-11-21 15:57:00,394.7200,394.7200,394.4200,394.4345,284264
197,2022-11-21 15:56:00,394.8000,394.8000,394.6300,394.7000,278622
198,2022-11-21 15:55:00,395.0600,395.1200,394.7700,394.8000,354828
199,2022-11-21 15:54:00,394.9000,395.0800,394.8300,395.0600,283090
200,2022-11-21 15:53:00,394.7500,394.9200,394.7400,394.9000,147763
201,2022-11-21 15:52:00,394.6300,394.9200,394.5900,394.7500,188787


,time,open,high,low,close,volume
0,2022-11-21 19:18:00,37.550,37.5500,37.5500,37.5500,206
1,2022-11-21 19:06:00,37.550,37.5500,37.5500,37.5500,1000
2,2022-11-21 18:46:00,37.550,37.5500,37.5500,37.5500,145
3,2022-11-21 18:43:00,37.550,37.5500,37.5500,37.5500,270
4,2022-11-21 18:01:00,37.550,37.5500,37.5500,37.5500,461
5,2022-11-21 17:51:00,37.550,37.5500,37.5500,37.5500,100
6,2022-11-21 17:00:00,37.550,37.5500,37.5500,37.5500,2000
7,2022-11-21 16:34:00,37.530,37.5300,37.5200,37.5200,4600
8,2022-11-21 16:30:00,37.550,37.5500,37.5400,37.5400,9599
9,2022-11-21 16:26:00,37.550,37.5500,37.5500,37.5500,7501


,time,open,high,low,close,volume
0,2022-11-21 16:00:00,48.7810,48.8000,48.7600,48.7600,5387
1,2022-11-21 15:59:00,48.7800,48.7800,48.7745,48.7745,874
2,2022-11-21 15:58:00,48.7700,48.7725,48.7700,48.7725,993
3,2022-11-21 15:56:00,48.7892,48.7892,48.7700,48.7700,559
4,2022-11-21 15:55:00,48.7601,48.7601,48.7601,48.7601,422
5,2022-11-21 15:54:00,48.7601,48.7900,48.7601,48.7900,2033
6,2022-11-21 15:52:00,48.7850,48.7850,48.7850,48.7850,212
7,2022-11-21 15:51:00,48.7800,48.7800,48.7700,48.7700,334
8,2022-11-21 15:50:00,48.7800,48.7850,48.7800,48.7850,8183
9,2022-11-21 15:46:00,48.7500,48.7700,48.7500,48.7700,597


In [17]:
# fix date, create returns and rate differences over 1d, 5d, 21d, 63d, 121d, 252d, 504d for all 4 variables
# then create signals by lagging evth by 1d
# then clean evth and fir xgb (around 30 features)

#### step 1:
build simple XGB model
#### step 2:
deploy this model via Cloud Run and static web app, fixed model
#### step 3
deploy model via Cloud Run and Flask with dynamic model, retrained daily